<a href="https://colab.research.google.com/github/RahaviSelvarajan/yolov5_colab/blob/main/SBI_YOLO_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!python --version

Python 3.7.13


##***Mount Google Drive***

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##***Mount Onedrive to get the dataset***

In [ ]:
!wget https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
!apt install ./rclone-v1.57.0-linux-amd64.deb

In [ ]:
!rclone config

In [ ]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

**Run the below cell only when onedrive gets disconnected in the middle of training and mount it again using the above command.**

In [ ]:
!fusermount -u /content/onedrive

##***Git Clone YOLOv5-Pytorch model***

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12530, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12530 (delta 7), reused 17 (delta 7), pack-reused 12511
Receiving objects: 100% (12530/12530), 11.63 MiB | 20.32 MiB/s, done.
Resolving deltas: 100% (8722/8722), done.


Install the required python packages

In [3]:
%cd yolov5
!pip install -r requirements.txt

/content/yolov5
     |████████████████████████████████| 596 kB 14.8 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


##***Training***

In [4]:
# Copy data configuration file into yolo directory
!cp /content/drive/MyDrive/Yolov5/sleepaid.yaml /content/yolov5/data/

In [ ]:
# Import libraries
import os
import glob
import shutil
from pathlib import Path

# Get the train and test dataset from different folds
def train_test_split(root_dir, test_fold):

    # Make directory for training and testing images
    train_imgs = Path('/content/yolov5/data/images/train')
    test_imgs = Path('/content/yolov5/data/images/test')

    if train_imgs.exists():
      shutil.rmtree(train_imgs)
    if test_imgs.exists():
      shutil.rmtree(test_imgs)

    os.mkdir(train_imgs)
    os.mkdir(test_imgs)

    # Make directory for target labels
    train_labels = Path('/content/yolov5/data/labels/train')
    test_labels = Path('/content/yolov5/data/labels/test')

    if train_labels.exists():
      shutil.rmtree(train_labels)
    if test_labels.exists():
      shutil.rmtree(test_labels)

    os.makedirs(train_labels)
    os.makedirs(test_labels)

    for j, image in enumerate(glob.iglob(os.path.join('/content/drive/MyDrive/Negatives', "*.jpg"))):
      if j <= 150:
        shutil.copy(image, train_imgs)
      else:
        shutil.copy(image, test_imgs)

    for i, folder in enumerate(os.listdir(root_dir)):
      folder_path = os.path.join(root_dir, folder)
      file_list = [image for image in glob.glob(os.path.join(folder_path, '*.jpg'))]

      # Get the train folds
      if i != test_fold:
        for image in file_list:
          shutil.copy(image, train_imgs)
          shutil.copy(os.path.splitext(image)[0] + '.txt', train_labels)

      # Get the test fold
      elif i == test_fold:
        for image in file_list:
          shutil.copy(image, test_imgs)
          shutil.copy(os.path.splitext(image)[0] + '.txt', test_labels)

for i in range(2, 3):
  train_test_split('/content/drive/MyDrive/Dataset', i)
  !python train.py --batch-size 8 --img-size 1920 --data 'sleepaid.yaml' --weights 'yolov5s.pt' --epochs 300 --cfg 'yolov5s.yaml'  \
                   --save-period 50 --patience 20 --hyp '/content/drive/MyDrive/Yolov5/hyp.yaml' --project '/content/drive/MyDrive/Yolov5/runs/train' 
  !mv /content/drive/MyDrive/Yolov5/runs/train/exp /content/drive/MyDrive/Yolov5/runs/train/results"$i"

train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=sleepaid.yaml, hyp=/content/drive/MyDrive/Yolov5/hyp.yaml, epochs=300, batch_size=8, imgsz=1920, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/Yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=20, freeze=[0], save_period=50, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-105-gd257c75 torch 1.10.0+cu111 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.